# xcube server loading places from geoDB

This notebook describes how to enable xcube server to load places from the [xcube geoDB](https://github.com/dcs4cop/xcube-geodb).

For this to work, the following requirements must be satisfied:

* xcube version must be >= 1.0.3
* The dedicated xcube plugin [xcube-geodb-places)](https://github.com/dcs4cop/xcube-geodb-places) must be installed:
  `cd ${projects}`
  `git clone https://github.com/dcs4cop/xcube-geodb-places.git`
  `cd xcube-geodb-places`
  `mamba activate xcube`
  `pip install -e .`
* in order to view the places in the xcube viewer, it must be installed. See [here](https://github.com/dcs4cop/xcube-viewer) for instructions.

## Configuration

Add the places you want to load from the xcube geoDB to the server configuration, and (re-)start xcube server.

A sample configuration can be copied and adapted from [here](https://github.com/dcs4cop/xcube-geodb-places/blob/main/xcube_places_plugin/config.yml.example).

### Explanation of the configuration items

#### clientId and clientSecret:

`GEODB_AUTH_CLIENT_ID:      <your_client_id>`

`GEODB_AUTH_CLIENT_SECRET:  <your_client_secret>`

If you are using xcube geoDB via EDC, your clientId and clientSecret are stored as environment variables. You can retrieve them by starting a Jupyter Notebook on EDC, and issuing the commands:

```
import os

os.environ('GEODB_AUTH_CLIENT_ID')
os.environ('GEODB_AUTH_CLIENT_SECRET')
```


#### Identifier:

Some arbitrary name for the group of places. Can be anything, but must not contain spaces.


#### Title:

The title for the group of places, as it appears in the viewer.


#### Query:

A [PostGREST](https://postgrest.org/en/stable/references/api/tables_views.html)-compliant query to the geoDB. This query determines the places to load from the geoDB. The example (`collection-name?limit=100&depth=gt.10&select=list,of,columns`) translates to "fetch the columns `list`, `of`, and `columns` from the first 100 rows from the collection `collection-name`, where the value of column `depth` is greater than 10". There are plenty of other possibilities to design queries, which are described in the PostGREST documentation.


#### DatasetRefs:

The place groups defined in the configuration will only be shown for the datasets listed here.


#### PropertyMapping:

These fields control the display: you can define which column of the collection determines the label, the color, or the description for the places. For example, if you have a collection with a column `feature-name`, you may want to put here `label: feature-name`.

### Demonstration of a sample configuration

In [ ]:
from xcube.webapi.viewer import Viewer

viewer = Viewer(server_config={
    "GeoDBConf": {
        "GEODB_API_SERVER_URL": "https://xcube-geodb.brockmann-consult.de",
        "GEODB_API_SERVER_PORT": 443,
        "GEODB_AUTH_AUD": "https://xcube-users.brockmann-consult.de/api/v2",
        "GEODB_AUTH_CLIENT_ID": "<your_client_id>",
        "GEODB_AUTH_CLIENT_SECRET": "<your_client_secret>",
        "PlaceGroups": [
            {
                "Identifier": "doors_BSWQDB_JOSS",
                "Title": "DOORS - BSWQDB for JOSS GE-UA 2016",
                "Query": 'doors_BSWQDB_1?select=station,"station depth [m]","sampling depth [m]","salinity [‰]","temperature [°c]",timestamp&cruise=eq.JOSS GE-UA 2016',
                "PropertyMapping": {
                    "label": "station"
                },
                "DatasetRefs": ["demo-cmems-oc-bs"]
            }
        ]
    },
    "DataStores": [
        {
            "Identifier": "doors-demo",
            "StoreId": "s3",
            "StoreParams": {
                "root": "doors-cubes/blacksea",
                "storage_options": {
                    "anon": False
                }
            },
            "Datasets": [
                {
                    "Identifier": "demo-cmems-oc-bs",
                    "Path": "CMEMS_OC_BS_4.zarr",
                    "TimeSeriesDataset": "demo-cmems-oc-bs-time-opt",
                    "Style": "default",
                    "AccessControl": {
                        "IsSubstitute": True
                    }
                }
            ]
        }
    ]

})

In [ ]:
viewer.info()